## Data Loading

In [ ]:
import sys
import os
import faiss
import numpy as np
import pandas as pd
from fais_functions import *

sys.path.append(os.path.abspath(".."))

from config import *


In [ ]:
df = load_embeddings_and_data(EMBEDEDINGS_FILEPATH_Mini_LML6_v2, CSV_FILEPATH_UNCHANGED_DATA)

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

## Search From Multiple Entries

In [ ]:
def recommend_from_multiple_wines(indices: list, df_vectors: pd.DataFrame, index, k: int = 5):
    selected_vectors = df_vectors.iloc[indices].to_numpy()
    query_vector = np.mean(selected_vectors, axis=0).reshape(1, -1) # Uśredniamy wektory, aby stworzyć jeden wektor zapytania reprezentujący wszystkie wybrane wina
    query_vector = query_vector.astype('float32')
    faiss.normalize_L2(query_vector)
    D, I = index.search(query_vector, k)
    
    return D, I

## Embeding from user's wine description

In [ ]:
def get_single_embedding(text, model=model, client=client):
    try:
        if not text:
            print("Text is null")
            return []
            
        text = text.replace("\n", " ")
        response = client.embeddings.create(input=[text], model=model)
        return response.data[0].embedding
        
    except Exception as e:
        print(f"Błąd przy generowaniu embeddingu: {e}")
        return []

In [ ]:
some_wine_description = "A rich and full-bodied red wine with notes of blackberry, plum, and a hint of oak. Perfect for pairing with grilled meats or hearty stews."

import openai
from private import API_KEY

client = openai.OpenAI(api_key=API_KEY)
MODEL = "text-embedding-3-small"
embedding = get_single_embedding(some_wine_description, model=MODEL, client=client)

client_number = "001"
client_embedings_file_path = f"/clients_embeddings/embeddings_open_ai_api_for_client{client_number}.npy"
append_embedding_to_file(client_embedings_file_path, embedding)